In [6]:
import os, glob

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import logit, loglogit, piecewiseFormula, nested
from biogeme.models import lognested
from biogeme.results import bioResults, pickle

In [7]:
df = pd.read_table("new_lpmc02.dat", index_col=0)

In [8]:
# create biogeme database

database = db.Database("LPMC",df)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

In [9]:
# Model

# Choice
chosenAlternative = travel_mode


#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)

CarOwn_2 = Beta('CarOwn_2',0,None,None,0)
CarOwn_3 = Beta('CarOwn_3',0,None,None,0)
CarOwn_4 = Beta('CarOwn_4',0,None,None,0)

LAMBDA = Beta('LAMBDA',1,None,None,0)

# parameters relevant to the nests
N_SM = Beta('N_SM',1,1,None, 0)
N_MOTOR = Beta('N_MOTOR',1,1,None, 0)


# socio-economic factors (interacting with Time)
Time_Age_1 = Beta('Time_Age_1', 0, None, None, 0)
Time_Age_2 = Beta('Time_Age_2', 0, None, None, 0)
Time_Age_3 = Beta('Time_Age_3', 0, None, None, 0)
Time_Age_4 = Beta('Time_Age_4', 0, None, None, 0)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving


cost_public_inc = DefineVariable('cost_public_inc', cost_transit*1.15, database)
dur_public = DefineVariable('dur_public', (dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int),database)
cost_driving = DefineVariable('cost_driving', (cost_driving_ccharge + cost_driving_fuel), database)

Opt1 = Constant1 + Total_TT1 * ((dur_walking) ** LAMBDA -1)/LAMBDA + Time_Age_1 * dur_walking * age
Opt2 = Constant2 + Total_TT2 * ((dur_cycling) ** LAMBDA -1)/LAMBDA+ CarOwn_2 * car_ownership +\
                    Time_Age_2 * dur_cycling * age
Opt3 = Constant3 + Cost * cost_public_inc + Total_TT3 * (dur_public ** LAMBDA -1)/LAMBDA + CarOwn_3 *\
                car_ownership + Time_Age_3 * dur_public * age
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * ((dur_driving) ** LAMBDA -1)/LAMBDA +\
                    CarOwn_4 * car_ownership + Time_Age_4 * dur_driving * age


V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1, 2: 1, 3: 1, 4: 1}


#Definitions of nests
N_SM = N_SM, [1, 2]
N_MOTOR = N_MOTOR, [3, 4]

nests = N_SM, N_MOTOR

In [10]:
# Model estimates


output_dir = "./model-nested-output"
filepath = os.path.join(output_dir, "logit_nested_lpmc_sm_motor_public_increased")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
# delete previously saved html and pickle
for file in glob.glob(f"{filepath}*"):
    os.remove(file)
    
# The choice model is a logit, with availability conditions
logprob = lognested(V, av, nests, chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = filepath
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

pandasResults

Nbr of observations: 5000
LL(0) =    -6931.472
LL(beta) = -3650.326
rho bar square = 0.471
Output file: ./model-nested-output/logit_nested_lpmc_sm_motor_public_increased.html


,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
CarOwn_2,-0.199,0,0.138,-1.44,0.15,0.148,-1.35,0.177
CarOwn_3,-0.24,0,0.138,-1.74,0.0815,0.16,-1.5,0.135
CarOwn_4,0.817,0,0.0898,9.09,0,0.0974,8.39,0
Constant2,-2.32,0,0.468,-4.97,6.8e-07,0.435,-5.34,9.43e-08
Constant3,2.03,0,0.236,8.59,0,0.23,8.83,0
Constant4,0.0197,0,0.403,0.0488,0.961,0.474,0.0414,0.967
Cost,-0.088,0,0.019,-4.64,3.51e-06,0.0237,-3.71,0.000206
LAMBDA,-0.00139,0,0.102,-0.0137,0.989,0.122,-0.0114,0.991
N_MOTOR,1.64,0,0.279,5.89,3.93e-09,0.338,4.86,1.18e-06
N_SM,1,1,0.222,4.51,6.35e-06,0.311,3.22,0.00128


In [11]:
# market shares with increased costs

sum_weights = database.data['Weights'].sum()
S = database.getSampleSize()
sample_normalized_weight = Weights * S / sum_weights


prob_walking = nested(V, av, nests, 1)
prob_cycling = nested(V, av, nests, 2)
prob_public = nested(V, av, nests, 3)
prob_car = nested(V, av, nests, 4)


simulate = {'Prob. walking': prob_walking,
            'Prob. cycling': prob_cycling,
            'Prob. public': prob_public,
            'Prob. car': prob_car,
            'Weighted prob. walking': sample_normalized_weight * prob_walking,
            'Weighted prob. cycling': sample_normalized_weight * prob_cycling,
            'Weighted prob. public': sample_normalized_weight * prob_public,
            'Weighted prob. car': sample_normalized_weight * prob_car
           }


biogeme  = bio.BIOGEME(database, simulate)
#biogeme.modelName = filepath

betas = biogeme.freeBetaNames
results = bioResults(pickleFile=f"{filepath}.pickle")

beta_values = results.getBetaValues()

# get simulated values from prev saved beta values
simulated_values = biogeme.simulate(beta_values)

# calculate market shares 
marketShare_walking = 100 * simulated_values['Weighted prob. walking'].mean()
marketShare_cycling = 100 * simulated_values['Weighted prob. cycling'].mean()
marketShare_public = 100 * simulated_values['Weighted prob. public'].mean()
marketShare_car = 100 * simulated_values['Weighted prob. car'].mean()

# conf interval
b = results.getBetasForSensitivityAnalysis(betas, size=100)
left, right = biogeme.confidenceIntervals(b, 0.9)

# display results
lst_marketShares = [marketShare_walking, marketShare_cycling, marketShare_public, marketShare_car]
temp_names = ["marketShare_walking", "marketShare_cycling", "marketShare_public", "marketShare_car"]

print("Predicted market shares:\n")
for i in range(len(temp_names)):
    l = left[f"Weighted prob. {temp_names[i].split('_')[1]}"].mean()*100
    r = right[f"Weighted prob. {temp_names[i].split('_')[1]}"].mean()*100
    print(f"{temp_names[i]}: {lst_marketShares[i]:.2f}% ({l:.2f}%, {r:.2f}%)")

Predicted market shares:

marketShare_walking: 17.59% (15.28%, 20.26%)
marketShare_cycling: 2.84% (2.24%, 7.93%)
marketShare_public: 36.43% (33.36%, 41.37%)
marketShare_car: 43.15% (35.14%, 45.46%)


In [12]:
beta_values

{'CarOwn_2': -0.1993888902362113,
 'CarOwn_3': -0.23988693357401222,
 'CarOwn_4': 0.8169483628265232,
 'Constant2': -2.3243561850487158,
 'Constant3': 2.027755719752533,
 'Constant4': 0.019661812248610605,
 'Cost': -0.08800899109684662,
 'LAMBDA': -0.0013938133073915972,
 'N_MOTOR': 1.6426979177515224,
 'N_SM': 1.0,
 'Time_Age_1': -0.051057218790012,
 'Time_Age_2': -0.028391246481041455,
 'Time_Age_3': -0.03456558824654881,
 'Time_Age_4': -0.043405325169158265,
 'Total_TT1': -2.8065802972269,
 'Total_TT2': -1.31777982833698,
 'Total_TT3': -0.6314042362124275,
 'Total_TT4': -1.0022740945930384}